# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [ ]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Uninstall flash-attention (the PyPI package name may vary)
!pip uninstall flash-attn -y
!pip uninstall xformers -y

!pip install flash-attn
!pip install xformers


# Clean up the notebook
clean_notebook()



In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

# Sign in to Hugging Face

1. If you haven't already done so, create a free HuggingFace account at https://huggingface.co and navigate to Settings, then Create a new API token, giving yourself write permissions

2. Press the "key" icon on the side panel to the left, and add a new secret:
`HF_TOKEN = your_token`

3. Execute the cell below to log in.

In [1]:
import os
from huggingface_hub import login

hf_token = os.environ.get('HF_TOKEN')

# Log into Hugging Face, adding credentials to git if needed
login(hf_token, add_to_git_credential=False)

In [19]:
# instruct models

LLAMA    = "meta-llama/Meta-Llama-3.1-8B-Instruct"
Deepseek ="deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


In [27]:
messages = [
    {"role": "system", "content": "You are a helpful assistant answer in thai language"},
    {"role": "user", "content": "เล่านิทานให้ที in thai language เกี่ยวกับ แมว และ สุนัข และ เต่า "}
  ]

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole family of models.

In [7]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

If the next cell gives you a 403 permissions error, then please check:
1. Are you logged in to HuggingFace? Try running `login()` to check your key works
2. Did you set up your API key with full read and write permissions?
3. If you visit the Llama3.1 page at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B, does it show that you have access to the model near the top?


In [8]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

In [9]:
# The model
model = AutoModelForCausalLM.from_pretrained(
    LLAMA, 
    device_map="auto", 
    quantization_config=quant_config,
    
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 5,591.5 MB


In [11]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [12]:
outputs = model.generate(inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

เล่านิทานให้ที<|eot_id|><|start_header_id|>assistant<|end_header_id|>

นิทานที่ฉันจะเล่าให้คุณฟังวันนี้คือเรื่อง "หมูสามตัว"

มีสามตัวหมูอยู่ในสวนหมู พวกมันชอบเล่นกันและกินอาหารได้เยอะ แต่หมูหนึ่งตัวมีชีวิตที่ไม่เหมือนกับหมูที่เหลือ มันชอบไปเดินเล่นในสวนและดูพืชพันธุ์ต่างๆ มากกว่าที่จะกินอาหาร

หนึ่งวัน หมูสามตัวนี้ไปพบกับหมูอื่นๆ ที่กำลังเล่นอยู่ในสวน "ทำไมคุณไม่ไปกินอาหาร?" หมูอื่นๆ ถาม "อาหารมันเยอะมากที่นี่" แต่หมูหนึ่งตัวนั้นไม่สนใจ "ฉันอยากไปดูพืชพันธุ์ต่างๆ มากกว่าที่จะกินอาหาร"

หมูอื่นๆ ก็ไม่เข้าใจว่าทำไมหมูหนึ่งตัวจะไม่ชอบกินอาหาร แต่พวกมันก็ไม่ได้บังคับหมูหนึ่งตัวให้ทำตามพวกมัน

วันเวลาผ่านไปและหมูสามตัวนี้ก็โตขึ้นและแข็งแรงขึ้น แต่หมูหนึ่งตัวนั้นไม่เคยเปลี่ยนแปลงความชอบของมัน "ฉันชอบไปดูพืชพันธุ์ต่างๆ มากกว่าที่จะกินอาหาร" มันพูด

สุดท้าย หมูสามตัวนี้ก็อายุได้ 10 ปี และหมูหน

In [13]:
# Clean up

del inputs, outputs, model
torch.cuda.empty_cache()

## A couple of quick notes on the next block of code:

I'm using a HuggingFace utility called TextStreamer so that results stream back.
To stream results, we simply replace:  
`outputs = model.generate(inputs, max_new_tokens=80)`  
With:  
`streamer = TextStreamer(tokenizer)`  
`outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)`

Also I've added the argument `add_generation_prompt=True` to my call to create the Chat template. This ensures that Phi generates a response to the question, instead of just predicting how the user prompt continues. Try experimenting with setting this to False to see what happens. You can read about this argument here:

https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-prompts

Thank you to student Piotr B for raising the issue!

In [24]:
# Wrapping everything in a function - and adding Streaming and generation prompts

def generate(model, messages):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=2500, streamer=streamer)
  del tokenizer, streamer, model, inputs, outputs
  torch.cuda.empty_cache()

In [25]:
generate(LLAMA, messages)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant answer in thai language<|eot_id|><|start_header_id|>user<|end_header_id|>

เล่านิทานให้ที เกี่ยวกับ แมว และ สุนัข และ เต่า<|eot_id|><|start_header_id|>assistant<|end_header_id|>

มีครั้งหนึ่ง 3 ตัวสัตว์อยู่ในสวน 

แมวหนึ่งตัว มีสีดำและขาว มีใจอยากเล่นและล่าเหยื่อ

สุนัขหนึ่งตัว มีสีแดง มีใจอยากเล่นและใช้กำลัง

เต่าหนึ่งตัว มีสีเหลือง มีใจอยากว่ายน้ำและพักผ่อน

หนึ่งวัน แมวและสุนัขได้พบกันในสวน แมวพูดว่า "ฉันสามารถล่าเหยื่อได้ไง" สุนัขพูดว่า "ฉันสามารถใช้กำลังได้ไง" เต่าพูดว่า "ฉันสามารถว่ายน้ำได้ไง"

แมวและสุนัขได้เล่นกันอยู่ในสวน แต่เต่าไม่กล้าเล่นเพราะกลัวจะถูกล่า

จนกระทั่งแมวและสุนัขได้ทำร้ายตัวเองกันเอง แมวได้ถูกแมวอีกตัวหนึ่งทำร้าย สุนัขได้ถูกสุนัขอีกตัวหนึ่งทำร้าย

เต่าได้เห็นเหตุการณ์นี้และพูดว่า "ฉันสามารถพักผ่อนได้ไง" แมวและสุนัขได้ฟังคำพูดของเต่าและได้เรียนรู้ว่า การพักผ่อนเป็นเรื่องสำคัญ

จากนั้น แมวและสุนัขได้เล่นกันอยู่ในส

In [28]:
generate(Deepseek, messages)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<｜begin▁of▁sentence｜>You are a helpful assistant answer in thai language<｜User｜>เล่านิทานให้ที in thai language เกี่ยวกับ แมว และ สุนัข และ เต่า <｜Assistant｜><think>
Okay, the user wants a story in Thai about cats, dogs, and turtles. I should make it engaging and suitable for all ages. Let me think of a simple plot that includes these animals and some adventures.

Maybe the animals can go on a journey together, facing some challenges. That way, I can include each of them and create a fun scenario. I'll set it in a village near a river with a temple, which is a common setting in Thai stories.

I'll introduce each animal with their own traits: the cat as clever, the dog as loyal, and the turtle as wise. Their interactions can show teamwork and friendship. Perhaps they come together when something happens, like finding a lost item or helping someone in need.

I should include some Thai vocabulary to make it authentic without making it too complex. The story should have a happy ending, sho